In [1]:
import pandas as pd
import numpy as np
import string
import nltk
from matplotlib import pyplot as plt
# from nltk.corpus import stopwords
# nltk.download("stopwords")
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import tree 
from sklearn import svm 
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, recall_score
nltk.download('stopwords')

import matplotlib
import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline
sns.set()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\llctt\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Using pandas ro load the dataset remotely
df = pd.read_csv("https://raw.githubusercontent.com/yqian000/OnlineReviewDetection/main/reviews.csv")

### Data preprocessing  
1. Remove punctuation  
2. Remove stopwords

In [3]:
'''
def clean_text(text):
    # remove punctuation
    text_not_punc = []
    i = 0
    for c in text:
        if c not in string.punctuation:
            text_not_punc.append(c)
    # Join characters into string.
    text_not_punc = ''.join(text_not_punc) 

    # remove stopwords
    text_not_punc = list(text_not_punc.split(" "))
    text_clean = []
    for word in text_not_punc:
        if word.lower() not in stopwords.words('english'):
            text_clean.append(word)
    return text_clean
'''

'\ndef clean_text(text):\n    # remove punctuation\n    text_not_punc = []\n    i = 0\n    for c in text:\n        if c not in string.punctuation:\n            text_not_punc.append(c)\n    # Join characters into string.\n    text_not_punc = \'\'.join(text_not_punc) \n\n    # remove stopwords\n    text_not_punc = list(text_not_punc.split(" "))\n    text_clean = []\n    for word in text_not_punc:\n        if word.lower() not in stopwords.words(\'english\'):\n            text_clean.append(word)\n    return text_clean\n'

In [4]:
# Create train and test dataset
X = df["text_"]
y = df["label"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)


In [5]:
def print_train_test_accuracy(model, train_pred, test_pred):
    train_accuracy = model.score(train_pred, y_train)
    test_accuracy = model.score(test_pred, y_test)
    print(f"Train Accuracy: {train_accuracy*100:.2f}%, Test Accuracy: {test_accuracy*100:.2f}%")

### Feature Extraction
note: should call transform only on the test set to use the same vocabulary as for the training data

#### CountVectorizer + TfidfTransformer

In [6]:
cv = CountVectorizer()
tfidf = TfidfTransformer()

# train set
cv_train_features = cv.fit_transform(X_train)
tfidf_train_features = tfidf.fit_transform(cv_train_features)

# test set
cv_test_features = cv.transform(X_test)
tfidf_test_features = tfidf.transform(cv_test_features)


### DecisionTree model

In [7]:
model = tree.DecisionTreeClassifier()
model.fit(tfidf_train_features, y_train)
print_train_test_accuracy(model, tfidf_train_features, tfidf_test_features)

Train Accuracy: 100.00%, Test Accuracy: 77.72%


### SVM model

In [8]:
model = svm.SVC()
model.fit(tfidf_train_features, y_train)
print_train_test_accuracy(model, tfidf_train_features, tfidf_test_features)

Train Accuracy: 98.75%, Test Accuracy: 91.96%


### KNN

In [9]:
new_df=df[['text_',"label"]]
new_df.loc[new_df["label"]=="CG","label"]=1
new_df.loc[new_df["label"]=="OR","label"]=0
X = new_df["text_"]
y = new_df["label"]

C:\Users\llctt\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\llctt\anaconda3\lib\site-packages\pandas\core\indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [10]:
corpus = []
for i in range(0,len(X)):
    review = re.sub('[^a-zA-Z]', ' ', new_df['text_'][i])
    review = review.lower()
    review = review.split()
    ps = PorterStemmer()
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)

In [11]:
cv = CountVectorizer(max_features = 1500)
X = cv.fit_transform(corpus).todense()
y = df.iloc[:,1].values


In [13]:

X_train, X_test, y_train, y_test = train_test_split(X , y, test_size = 0.20, random_state = 0)

In [35]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [36]:
y_pred = classifier.predict(X_train)
Accuracy_Score = accuracy_score(y_train, y_pred)
print("Train accuracy Score is :", str(round(Accuracy_Score*100.0,2))+"%")


Train accuracy Score is : 22.62%


In [37]:

y_pred = knn.predict(X_test)
Accuracy_Score = accuracy_score(y_test, y_pred)
print("Test accuracy Score is :", str(round(Accuracy_Score*100.0,2))+"%")


Test accuracy Score is : 55.27%


### Naive Bayes Model

In [38]:
classifier = GaussianNB()
classifier.fit(X_train, y_train)
print(classifier.fit(X_train, y_train))

GaussianNB()


In [39]:
y_pred = classifier.predict(X_train)
Accuracy_Score = accuracy_score(y_train, y_pred)
print("Test accuracy Score is :", str(round(Accuracy_Score*100.0,2))+"%")

Test accuracy Score is : 22.62%


In [40]:
y_pred = classifier.predict(X_test)
Accuracy_Score = accuracy_score(y_test, y_pred)
print("Train accuracy Score is :", str(round(Accuracy_Score*100.0,2))+"%")


Train accuracy Score is : 18.96%
